## 1. 파이프라인을 사용하지 않고 타이타닉 데이터 분석하기
범주형, 숫자형 칼럼에 대한 전처리
모델을 학습시키고 적용하는 과정을 40줄에 걸쳐서 해야함.

In [2]:
import pandas as pd
import numpy as np

In [9]:
from catboost.datasets import titanic
titanic_train, titanic_test = titanic()

print(titanic_train.head(3))

RuntimeError: ('failed to download from %s', ('https://storage.mds.yandex.net/get-devtools-opensource/233854/titanic.tar.gz',))

In [7]:
from catboost.datasets import titanic

In [8]:
train, test = titanic()

RuntimeError: ('failed to download from %s', ('https://storage.mds.yandex.net/get-devtools-opensource/233854/titanic.tar.gz',))

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
### 결측값이 있는 컬럼명 출력 ###
[col for col in train.columns if train[col].isna().sum() >0]

In [ ]:
# 1. 다양한 형태의 컬럼 (숫자, 문자)
# 2. 다양한 정보 (ID, cardinality, parsible, ... )
# 3. 결측 (숫자결측, 문자결측)

In [ ]:
##### 2. 1st draft 모델링 #####

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
ycol = ["Survived"]
xcol = [col for col in train.columns if col not in ycol]

In [ ]:
X = train[xcol]
y = train[ycol]

In [ ]:
model = RandomForestClassifier()
model.fit(X, y)

In [ ]:
#### 3. One-Hot encoding ####

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = [col for col in X.columns if col not in num_cols]

In [ ]:
enc = OneHotEncoder(sparse=False)
enc.fit(X[cat_cols])

In [ ]:
new_cat_X = enc.transform(X[cat_cols])
new_X = np.concatenate([new_cat_X, X[num_cols].to_numpy()], axis=1)

In [ ]:
pd.DataFrame(new_X)

In [ ]:
model = RandomForestClassifier()
model.fit(new_X, y)

In [ ]:
#### 4. 결측값 처리 ####

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer()
imp.fit(new_X)

In [ ]:
new_X2 = imp.transform(new_X)

In [ ]:
model = RandomForestClassifier()
model.fit(new_X2, y)

In [ ]:
fitted_value = model.predict(new_X2)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(fitted_value, y)

In [ ]:
test_pred = model.predict(test[xcol])
accuracy_score(test_pred, y)

In [ ]:
test_cat = enc.transform(test[cat_cols])

In [ ]:
# 6. Name, ticket 컬럼 삭제 #

In [ ]:
cat_cols = [col for col in X.columns if col not in num_cols + ['Name','Ticket','Cabin']]

In [ ]:
# 7. 전처리 다시 #

In [ ]:
enc = OneHotEncoder(sparse=False)
enc.fit(X[cat_cols])

In [ ]:
new_new_X = np.concatenate([enc.transform(X[cat_cols]), X[num_cols].to_numpy()], axis=1)

In [ ]:
imp = SimpleImputer()
imp.fit(new_new_X)

In [ ]:
new_new_new_X = imp.transform(new_new_X)

In [ ]:
# 8. 2nd draft 모델링

In [ ]:
model = RandomForestClassifier()
model.fit(new_new_new_X, y)

In [ ]:
fitted_value = model.predict(new_new_new_X)

In [ ]:
accuracy_score(fitted_value, y)

In [ ]:
# 9. test set 예측 

In [ ]:
new_test = enc.transform(test[cat_cols])

In [ ]:
new_new_test = np.concatenate([new_test, test[num_cols].to_numpy()], axis=1)

In [ ]:
new_new_new_test = imp.transform(new_new_test)

In [ ]:
# 9. test set 예측값 도출 #

In [ ]:
test_pred = model.predict(new_new_new_test)

In [ ]:
test_pred